## Exploração dos dados

In [1]:
import pandas as pd
import numpy as np
from rich import print

In [2]:
df = pd.read_csv('data/routes_base.csv')

In [3]:
df.head()

,route_id,schedule_date,car_type,route_distance_km,route_minutes,total_orders
0,562510,2019-06-29 08:00:00,Particular,79.149,532.93,55
1,563400,2019-06-29 15:45:00,Moto,22.391,104.85,4
2,563570,2019-06-29 15:45:00,Moto,62.106,142.81,7
3,563750,2019-06-29 15:45:00,Moto,46.681,102.24,6
4,564130,2019-06-30 08:00:00,Fiorino S.C.,20.151,68.22,2


In [4]:
#Avaliando os dados
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1778 entries, 0 to 1777
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   route_id           1778 non-null   int64  
 1   schedule_date      1778 non-null   object 
 2   car_type           1778 non-null   object 
 3   route_distance_km  1778 non-null   float64
 4   route_minutes      1778 non-null   float64
 5   total_orders       1778 non-null   int64  
dtypes: float64(2), int64(2), object(2)
memory usage: 83.5+ KB


In [5]:
#Verificando os tipos
df.dtypes

route_id               int64
schedule_date         object
car_type              object
route_distance_km    float64
route_minutes        float64
total_orders           int64
dtype: object

In [6]:
#Colunas com valores faltantes
print("[bold]Colunas com valores Faltantes: \n {0} \n[/bold]".format(
    df.columns[df.isnull().any()].values))

In [7]:
#linhas duplicadas
print("[bold]Duplicadas: {0} ".format(
        df.duplicated().sum()))

In [8]:
#Verificando as linhas nulas
df.nunique()

route_id             1778
schedule_date         132
car_type                8
route_distance_km    1742
route_minutes        1738
total_orders          105
dtype: int64

## Transfomando os dados

In [9]:
#Convertendo para o formato data
df['schedule_date'] = pd.to_datetime(df['schedule_date'])

In [10]:
df['planning_routes'] = np.where(df['schedule_date'].dt.hour <= 10, 'primeira saída', 'segunda saída')

In [11]:
df.head()

,route_id,schedule_date,car_type,route_distance_km,route_minutes,total_orders,planning_routes
0,562510,2019-06-29 08:00:00,Particular,79.149,532.93,55,primeira saída
1,563400,2019-06-29 15:45:00,Moto,22.391,104.85,4,segunda saída
2,563570,2019-06-29 15:45:00,Moto,62.106,142.81,7,segunda saída
3,563750,2019-06-29 15:45:00,Moto,46.681,102.24,6,segunda saída
4,564130,2019-06-30 08:00:00,Fiorino S.C.,20.151,68.22,2,primeira saída


In [12]:
df['km_pedido'] = df['route_distance_km']/df['total_orders']
df['min_pedido'] = df['route_minutes']/df['total_orders']

In [13]:
df['km_pedido'] = df['km_pedido'].astype(float).round(2)
df['min_pedido'] = df['min_pedido'].astype(float).round(2)

In [14]:
df.head()

,route_id,schedule_date,car_type,route_distance_km,route_minutes,total_orders,planning_routes,km_pedido,min_pedido
0,562510,2019-06-29 08:00:00,Particular,79.149,532.93,55,primeira saída,1.44,9.69
1,563400,2019-06-29 15:45:00,Moto,22.391,104.85,4,segunda saída,5.60,26.21
2,563570,2019-06-29 15:45:00,Moto,62.106,142.81,7,segunda saída,8.87,20.40
3,563750,2019-06-29 15:45:00,Moto,46.681,102.24,6,segunda saída,7.78,17.04
4,564130,2019-06-30 08:00:00,Fiorino S.C.,20.151,68.22,2,primeira saída,10.08,34.11


### Dicionário variáveis criadas
 - **planning_routes** {Indica o planejamento das rotas **primeira saída** (quando horário agendado da rota é menor que 10am) e **segunda saída** (quando horário agendado da rota é maior que 10am)}
 - **km_pedido** {Indicador total de quilômetros percorridos necessários para entregar um número X de Pedidos}
 - **min_pedido** {Indicado total de minutos necessários para entregar um número X de Pedidos}

### Qual o planejamento que possui maior concentração de pedidos?

In [15]:
df_plan = df[['planning_routes', 'total_orders']].groupby('planning_routes').sum().reset_index()
df_plan

,planning_routes,total_orders
0,primeira saída,47227
1,segunda saída,6940


Os pedidos estão mais concetrados na primeira saída. 

### Classificação dos indicadores de km/pedido e Min/pedido por tipo de veículo 

In [16]:
df_indic = df[['car_type', 'km_pedido', 'min_pedido']].groupby('car_type').mean().round(2).reset_index()
df_indic

,car_type,km_pedido,min_pedido
0,Bloqueado,4.65,16.47
1,Fiorino,2.48,11.80
2,Fiorino S.C.,4.60,16.60
3,Fiorino Sr,1.26,8.84
4,Hr,50.92,47.52
5,Moto,11.92,24.53
6,Particular,2.47,12.36
7,Próprio,11.15,30.27


Segundo os indicadores as rotas mais densas estão com os tipos:
 - **Hr** (Km/Pedido = 50.92 Min/Pedido = 47.52)
 - **Próprio** (Km/Pedido = 11.15 Min/Pedido = 30.27)
 - **Moto** (Km/Pedido = 11.92 Min/Pedido = 24.53)

## Escrevendo os Dataframes

In [17]:
with pd.ExcelWriter('data/case_kpi.xlsx') as writer:
    df.to_excel(writer, sheet_name='dados_transformados', index=False)
    df_plan.to_excel(writer, sheet_name='analise_planejamento', index=False)
    df_indic.to_excel(writer, sheet_name='analise_indicadores', index=False)